This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker import NNNodeBenchmarkerSSL
from graph_world.models.basic_gnn import GCN, SuperGAT
from torch_geometric.datasets import Planetoid, KarateClub, FakeDataset
from torch_geometric.transforms import RandomNodeSplit

from graph_world.self_supervised_learning.pretext_tasks.contrastive_based_different_scale import *


In [19]:
# Get dataset
dataset = FakeDataset(num_graphs = 1, avg_num_nodes = 100, num_channels = 16, num_classes = 4)[0]
dataset = RandomNodeSplit(split = "random", num_test = 20, num_val = 20, num_train_per_class=2)(dataset)

In [23]:
# Parameter setup (for cora)
benchmark_params = {
    'downstream_epochs' : 200,
    'pretext_epochs' : 200,
    'downstream_lr' : 3e-4,
    'pretext_lr' : 3e-4,
}

h_params = {
    'in_channels' : dataset.x.shape[1],
    'hidden_channels' : 128,
    'num_layers' : 2,
    'dropout' : 0.5,
}

pretext_params = {
    'cluster_ratio': 0.1, 
    'temperature' : 10, 
    'alpha': 0.5
}

generator_config = {
    'num_clusters' : 4,
}

pretext_task = GraphInfoClust
training_scheme = 'JL'

In [24]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerSSL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params,
                pretext_task=pretext_task, pretext_params = pretext_params, training_scheme=training_scheme)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

([0.6926726698875427,
  0.6907422542572021,
  0.6897557973861694,
  0.6861498951911926,
  0.686747133731842,
  0.6831564903259277,
  0.6842512488365173,
  0.6786571741104126,
  0.6786713600158691,
  0.6825772523880005,
  0.6780487298965454,
  0.6697288751602173,
  0.6809214353561401,
  0.66725754737854,
  0.6593004465103149,
  0.6635448932647705,
  0.6628794074058533,
  0.6532027721405029,
  0.6657019853591919,
  0.6599697470664978,
  0.660569429397583,
  0.6675618886947632,
  0.6410475969314575,
  0.6422056555747986,
  0.6375619769096375,
  0.6373186111450195,
  0.6391404867172241,
  0.6274549961090088,
  0.6212627291679382,
  0.6276535987854004,
  0.6192869544029236,
  0.6223033666610718,
  0.5870882272720337,
  0.660666286945343,
  0.6308118104934692,
  0.5774906873703003,
  0.6407903432846069,
  0.6230026483535767,
  0.5877243280410767,
  0.5901409387588501,
  0.6287999749183655,
  0.5896733999252319,
  0.6002299189567566,
  0.5965181589126587,
  0.6292098164558411,
  0.65030926465